In [42]:
#Downloading the dependencies 

from bs4 import BeautifulSoup #library for web scraping
import requests  # library to handle requests
import json  # library to handle JSON files
import xml
import pandas as pd #Python library data manipulation and analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



In [103]:

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

In [104]:
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

In [106]:
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
df

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


In [108]:
not_assigned = 'Not assigned'
not_assigned_row = df[ (df.Borough == not_assigned) & (df.Neighborhood == not_assigned) ]
not_assigned_row.head(), not_assigned_row.shape
df.drop(not_assigned_row.index, inplace=True)
df.dropna(inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [109]:

def neighborhood_list(grouped):    
    if( len(grouped) == 1 ):
        # only one line under the postal code assign the Borough as Neighborhood
        borough = grouped['Borough'].tolist()[0] 
        neighborhood = grouped['Neighborhood'].tolist()[0] 
        if( neighborhood) == not_assigned:
            return borough
        else:
            return neighborhood
    else:
        # transform grouped Neighborhood as single value separated with commas
        return ', '.join(sorted(grouped['Neighborhood'].tolist())) 
                    
grp = df.groupby(['PostalCode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [110]:
df2

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood]], Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [111]:
!pip install geocoder


    100% |################################| 102kB 2.9MB/s ta 0:00:01
    100% |################################| 71kB 2.9MB/s eta 0:00:01
    100% |################################| 829kB 1.2MB/s eta 0:00:01
    100% |################################| 92kB 3.5MB/s eta 0:00:01
    100% |################################| 143kB 3.5MB/s eta 0:00:01
    100% |################################| 61kB 3.6MB/s eta 0:00:01
    100% |################################| 143kB 3.4MB/s eta 0:00:01
    100% |################################| 153kB 3.4MB/s eta 0:00:01
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /home/spark/shared/.cache/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
Successfully built future


In [121]:
import geocoder # import geocoder

latitude = []
longitude = []
pcodes = df['PostalCode'].values

for postal_code in pcodes:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    
df['Latitude'] = latitude
df['Longitude'] = longitude

df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.753259,-79.329656
4,M4A,North York,Victoria Village,43.725882,-79.315572
5,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
6,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
7,M6A,North York,Lawrence Heights,43.718518,-79.464763
8,M6A,North York,Lawrence Manor,43.718518,-79.464763
9,M7A,Queen's Park,Not assigned,43.662301,-79.389494
11,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
12,M1B,Scarborough,Rouge,43.806686,-79.194353
13,M1B,Scarborough,Malvern,43.806686,-79.194353


In [113]:
# url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# source = requests.get(url).text

# soup = BeautifulSoup(source, 'lxml')#Beautiful Soup to Parse the url page

# table = soup.find('table') #Finds the required table area

# # #List initialization to collect the Postalcodes, Boroughs and Neighborhoods
# # postalcode=[]
# # borough=[]
# # neighborhood=[]
# x = table.tbody
# list1 = []
# for tr in x.find_all('tr'):
#     tds=tr.find_all('td')
#     for item in tds:
#         list1.append(item.text)
        

In [60]:
# postalcode = list1[0::3]
# borough = list1[1::3]
# neighborhood = list1[2::3]

In [61]:
# df=pd.DataFrame({'PostalCode':postalcode,
#                  'Borough':borough,
#                  'Neighborhood':neighborhood})

In [122]:
lat_lng_coords = None
while(lat_lng_coords is None):
    g = geocoder.google('Toronto, Ontario')
    lat_lng_coords = g.latlng

map_toronto = folium.Map(location=[lat_lng_coords[0], lat_lng_coords[1]], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [123]:
df2 = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [124]:
lat_lng_coords = None
while(lat_lng_coords is None):
    g = geocoder.google('Toronto, Ontario')
    lat_lng_coords = g.latlng

map_toronto = folium.Map(location=[lat_lng_coords[0], lat_lng_coords[1]], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto